In [5]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import asyncio
import json

# Import your modified run_finance_agent function
from your_finance_agent_module import run_finance_agent

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class QueryInput(BaseModel):
    query: str

async def event_generator(query):
    async for state in run_finance_agent(query):
        yield f"data: {json.dumps(state)}\n\n"
    yield "data: [DONE]\n\n"

@app.post("/query")
async def process_query(query_input: QueryInput):
    return StreamingResponse(event_generator(query_input.query), media_type="text/event-stream")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
import streamlit as st
import asyncio
import sys
from typing import Dict, Any

# Import your updated finance agent function
from your_finance_agent_module import run_finance_agent

st.title("Finance Agent Query Interface")

query = st.text_input("Enter your query:")

if st.button("Submit"):
    if query:
        st.write(f"Processing query: {query}")
        
        graph_placeholder = st.empty()
        results_placeholder = st.empty()
        final_response_placeholder = st.empty()

        def update_graph(current_node: str):
            nodes = ["subgraph_router", "execute_subgraphs", "aggregate_results", "end"]
            graph = " -> ".join([f"[{node}]" if node == current_node else node for node in nodes])
            graph_placeholder.text(f"Current State: {graph}")

        async def run_query():
            try:
                async for state in run_finance_agent(query):
                    st.write(f"Received state: {state['current_node']}")
                    update_graph(state['current_node'])
                    
                    if 'state' in state:
                        results_placeholder.json(state['state'])
                    
                    if state['current_node'] == 'end':
                        final_response = state.get('final_response', 'No final response available.')
                        final_response_placeholder.markdown(f"**Final Response:**\n\n{final_response}")
                
                st.success("Query processing completed!")
            except Exception as e:
                st.error(f"An error occurred: {str(e)}")
                st.exception(e)

        asyncio.run(run_query())
    else:
        st.warning("Please enter a query.")

st.markdown("""
## How to use this app:
1. Enter your finance-related query in the text box above.
2. Click the 'Submit' button to process your query.
3. Watch the graph update to show the current state of processing.
4. View the detailed results and final response below the graph.
""")

st.subheader("Debug Information")
st.write(f"Streamlit version: {st.__version__}")
st.write(f"Python version: {sys.version}")

In [ ]:
import streamlit as st
import asyncio
import sys
from finance_agent_backend import run_finance_agent

st.set_page_config(layout="wide")
st.title("Finance Agent Query Interface")

query = st.text_input("Enter your query:")

if st.button("Run Query"):
    if query:
        st.write(f"Processing query: {query}")
        
        # Create placeholders
        progress_bar = st.progress(0)
        status = st.empty()
        current_node = st.empty()
        state_display = st.expander("Current State (click to expand)")
        final_response = st.empty()

        async def run_query():
            try:
                step = 0
                total_steps = 6  # Based on the number of yield statements in run_finance_agent
                async for state in run_finance_agent(query):
                    step += 1
                    
                    # Update progress
                    progress_bar.progress(min(step / total_steps, 1.0))
                    
                    # Display current state
                    status.text(f"Status: Processing step {step}/{total_steps}")
                    current_node.text(f"Current node: {state['current_node']}")
                    
                    with state_display:
                        st.json(state['state'])
                    
                    if state['current_node'] == 'end':
                        final_response.markdown(f"**Final Response:** {state.get('final_response', 'No final response available.')}")
                
                status.text("Query processing completed!")
                progress_bar.progress(1.0)
            
            except Exception as e:
                st.error(f"An error occurred: {str(e)}")
                st.exception(e)

        # Run the async function
        asyncio.run(run_query())
    else:
        st.warning("Please enter a query.")

st.markdown("""
## How to use this app:
1. Enter your finance-related query in the text box above.
2. Click the 'Run Query' button to process your query.
3. Watch the progress and current state update in real-time.
4. Click on the 'Current State' expander to see detailed state information at each step.
5. The final response will be displayed when processing is complete.
""")

st.sidebar.subheader("Debug Information")
st.sidebar.text(f"Streamlit version: {st.__version__}")
st.sidebar.text(f"Python version: {sys.version}")

In [ ]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def run_finance_agent(query: str) -> AsyncGenerator[Dict[str, Any], None]:
    logger.info(f"Starting run_finance_agent with query: {query}")
    graph = create_finance_agent_graph()
    logger.debug(f"Created graph: {graph}")

    initial_state = AgentState(query=query)
    logger.debug(f"Initial state: {initial_state}")

    try:
        result = await graph.ainvoke(initial_state)
        logger.debug(f"Graph execution result: {result}")
        
        # Yield states based on your actual implementation
        yield {
            "current_node": "graph_db_router",
            "state": result
        }
        yield {
            "current_node": "query_graph_db",
            "state": result
        }
        yield {
            "current_node": "expand_queries",
            "state": result
        }
        yield {
            "current_node": "subgraph_router",
            "state": result
        }
        yield {
            "current_node": "execute_subgraphs",
            "state": result
        }
        yield {
            "current_node": "aggregate_results",
            "state": result
        }
        
        # Final state
        yield {
            "current_node": "end",
            "state": result,
            "final_response": result.get('final_response', 'No final response available.')
        }
    except Exception as e:
        logger.exception(f"An error occurred during graph processing: {e}")
        yield {
            "current_node": "error",
            "state": {"error": str(e)}
        }

    logger.info("Graph processing completed")

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1:
        query = " ".join(sys.argv[1:])
    else:
        query = "Analyze Apple Inc's relationship with its key suppliers."
    
    async def main():
        async for state in run_finance_agent(query):
            print(f"Current Node: {state['current_node']}")
            print(f"State: {state['state']}")
            print("---")
    
    asyncio.run(main())

In [ ]:
import asyncio
from typing import Dict, Any, AsyncGenerator
import logging
from langgraph.graph import Graph, StateGraph
from your_graph_module import create_finance_agent_graph, AgentState

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def run_finance_agent(query: str) -> AsyncGenerator[Dict[str, Any], None]:
    logger.info(f"Starting run_finance_agent with query: {query}")
    graph = create_finance_agent_graph()
    logger.debug(f"Created graph: {graph}")

    initial_state = AgentState(query=query)
    logger.debug(f"Initial state: {initial_state}")

    current_state = initial_state
    try:
        for node in graph.nodes:
            logger.info(f"Executing node: {node.__name__}")
            async for node_result in node(current_state):
                yield node_result
                current_state = node_result['state']
        
        # Final state
        yield {
            "current_node": "end",
            "state": current_state,
            "final_response": current_state.get('final_response', 'No final response available.')
        }
    except Exception as e:
        logger.exception(f"An error occurred during graph processing: {e}")
        yield {
            "current_node": "error",
            "state": {"error": str(e)}
        }

    logger.info("Graph processing completed")

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1:
        query = " ".join(sys.argv[1:])
    else:
        query = "Analyze Apple Inc's relationship with its key suppliers."
    
    async def main():
        async for state in run_finance_agent(query):
            print(f"Current Node: {state['current_node']}")
            print(f"State: {state['state']}")
            print("---")
    
    asyncio.run(main())

In [ ]:
import asyncio
from typing import Dict, Any, AsyncGenerator
import logging
from langgraph.graph import Graph, StateGraph
from your_graph_module import create_finance_agent_graph, AgentState

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def run_finance_agent(query: str) -> AsyncGenerator[Dict[str, Any], None]:
    logger.info(f"Starting run_finance_agent with query: {query}")
    graph = create_finance_agent_graph()
    logger.debug(f"Created graph: {graph}")

    state = AgentState(query=query)
    logger.debug(f"Initial state: {state}")

    try:
        for node in graph.nodes:
            logger.info(f"Executing node: {node.__name__}")
            async for node_result in node(state):
                # Update the state with the result from the node
                state.update(node_result['state'])
                yield {
                    "current_node": node.__name__,
                    "state": state.copy()  # Return a copy to avoid potential side effects
                }
        
        # Final state
        yield {
            "current_node": "end",
            "state": state.copy(),
            "final_response": state.get('final_response', 'No final response available.')
        }
    except Exception as e:
        logger.exception(f"An error occurred during graph processing: {e}")
        yield {
            "current_node": "error",
            "state": {"error": str(e)}
        }

    logger.info("Graph processing completed")

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1:
        query = " ".join(sys.argv[1:])
    else:
        query = "Analyze Apple Inc's relationship with its key suppliers."
    
    async def main():
        async for state in run_finance_agent(query):
            print(f"Current Node: {state['current_node']}")
            print(f"State: {state['state']}")
            print("---")
    
    asyncio.run(main())

In [ ]:
import asyncio
from typing import Dict, Any, AsyncGenerator
import logging
from langgraph.graph import Graph, StateGraph
from your_graph_module import create_finance_agent_graph, AgentState

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

async def run_finance_agent(query: str) -> AsyncGenerator[Dict[str, Any], None]:
    logger.info(f"Starting run_finance_agent with query: {query}")
    graph = create_finance_agent_graph()
    logger.debug(f"Created graph: {graph}")

    state = AgentState(query=query)
    logger.debug(f"Initial state: {state}")

    try:
        for node in graph.nodes:
            logger.info(f"Executing node: {node.__name__}")
            async for node_result in node(state):
                # Update the state with the result from the node
                state.update(node_result['state'])
                yield {
                    "current_node": node.__name__,
                    "state": state.copy()  # Return a copy to avoid potential side effects
                }
        
        # Final state
        yield {
            "current_node": "end",
            "state": state.copy(),
            "final_response": state.get('final_response', 'No final response available.')
        }
    except Exception as e:
        logger.exception(f"An error occurred during graph processing: {e}")
        yield {
            "current_node": "error",
            "state": {"error": str(e)}
        }

    logger.info("Graph processing completed")

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1:
        query = " ".join(sys.argv[1:])
    else:
        query = "Analyze Apple Inc's relationship with its key suppliers."
    
    async def main():
        async for state in run_finance_agent(query):
            print(f"Current Node: {state['current_node']}")
            print(f"State: {state['state']}")
            print("---")
    
    asyncio.run(main())